In [9]:
import torch
from brt._C.router import (
    generate_indices_and_loads,
    dispatch_with_indices_and_loads,
    split_fused_cells_to_paths
)

path_num = 4
gates = torch.randn((8, path_num)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

hot_mask = (
    torch.zeros(gates.size(0), path_num, dtype=torch.int32, device=gates.device,)
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
# supported_capacities = None

route_indices, dst_loads = generate_indices_and_loads(
    hot_mask, supported_capacities, capacity_padding=True, is_dst_index=True
)
print(route_indices)
print(dst_loads)


tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 2, 0],
        [1, 0, 0, 0],
        [2, 0, 0, 0],
        [0, 0, 0, 2],
        [3, 0, 0, 0],
        [4, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([4, 0, 2, 2], device='cuda:0', dtype=torch.int32)


In [10]:
in_data = torch.randn((8, path_num)).cuda()
print(in_data)
print(route_indices)
print(dst_loads)
(out_data_1,) = dispatch_with_indices_and_loads(in_data, route_indices, dst_loads)
print(out_data_1)
split_data = split_fused_cells_to_paths(out_data_1, dst_loads)
print(split_data)

out_data_1, tags = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads, tag_generating=True
)
split_data = split_fused_cells_to_paths(out_data_1, dst_loads, tags=tags)
print(split_data)

out_data_1, tags = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads, max_path_padding=True, tag_generating=True
)
split_data = split_fused_cells_to_paths(out_data_1, dst_loads, max_path_padding=True,tags=tags)
print(split_data)# print(out_data_1)

out_data_2 = dispatch_with_indices_and_loads(
    in_data, route_indices, dst_loads, is_1d_routing=False
)
print(out_data_2)
# torch.allclose(out_data_1[0], out_data_2)


tensor([[-3.0107e-01, -1.6857e-01,  1.1110e+00, -3.9645e-01],
        [-1.7685e+00, -5.3793e-01,  5.7216e-01, -1.8304e+00],
        [-1.4158e+00,  2.0783e+00,  3.2193e-01,  6.9651e-01],
        [ 4.0995e-01, -1.1471e-01,  1.0525e+00, -4.8346e-01],
        [ 1.1637e-01,  1.8467e-01, -6.4796e-01,  2.9781e-01],
        [ 1.4768e-04, -1.6258e+00, -5.3404e-01, -1.1788e+00],
        [-5.3572e-01, -4.7726e-01,  3.1614e-01,  3.7010e-01],
        [ 9.2405e-01, -3.0598e-01, -3.4310e-01, -2.8236e-01]], device='cuda:0')
tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 2, 0],
        [1, 0, 0, 0],
        [2, 0, 0, 0],
        [0, 0, 0, 2],
        [3, 0, 0, 0],
        [4, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([4, 0, 2, 2], device='cuda:0', dtype=torch.int32)
tensor([[ 4.0995e-01, -1.1471e-01,  1.0525e+00, -4.8346e-01],
        [ 1.1637e-01,  1.8467e-01, -6.4796e-01,  2.9781e-01],
        [-5.3572e-01, -4.7726e-01,  3.1614e-01,  3.7010e-01],
        [ 9.2405e-01, -3.0598e